In [0]:
df = spark.table("databricks_playground.default.housing_t")
display(df)

In [0]:
### getting some info from the table
print(df.count)
print(df.columns)
print(df.dtypes)

In [0]:
train_df, test_df = df.randomSplit([0.99, 0.01], seed=42)

In [0]:
from databricks import automl
summary=automl.regress(train_df,target_col='median_house_value',timeout_minutes=5)

In [0]:
import mlflow

# URI of the best MLflow run model
model_uri = f"runs:/{summary.best_trial.mlflow_run_id}/model"

# Convert MLflow model to a Spark UDF
predict = mlflow.pyfunc.spark_udf(spark, model_uri)

# Apply the model to the test dataframe and add predictions
pred_df = test_df.withColumn("prediction", predict(test_df.drop("median_house_value").columns))

# Display the results
display(pred_df)


In [0]:
rmse_evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="median_house_value", metricName="rmse")
rmse = rmse_evaluator.evaluate(pred_df)
print(f"RMSE on test dataset: {rmse:.3f}")